## Initialization

In [7]:
#import libraries
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#load dataset
train = pd.read_csv(r"C:\Users\LW130003\Documents\GitHub\spooky-author-identification/train.csv")
test = pd.read_csv(r"C:\Users\LW130003\Documents\GitHub\spooky-author-identification/test.csv")
train['corpus'] = 'text'
test['corpus'] = 'text'
train.head()

,id,text,author,corpus
0,id26305,"This process, however, afforded me no means of...",EAP,text
1,id17569,It never once occurred to me that the fumbling...,HPL,text
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,text
3,id27763,How lovely is spring As we looked from Windsor...,MWS,text
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,text


In [8]:
print('Number of rows: {}'.format(train.shape[0]))
print('Number of columns: {}'.format(train.shape[1]))

Number of rows: 19579
Number of columns: 4


## Natural Language Processing

I have created two codes to create train and test corpus. First, we will measure the time they need before extend it to whole dataset.

In [9]:
#method1
import time
t0 = time.time()

from nltk.corpus import wordnet
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
from nltk.stem.wordnet import WordNetLemmatizer
lemm = WordNetLemmatizer()

#train corpus
#for i in np.arange(train.shape[0]):
for i in np.arange(10):
    text = train.text.values[i] #extract text from data set train
    text = re.sub('[^a-zA-Z0-9]', ' ', text) #remove punctuation

    #tokenize
    text = nltk.word_tokenize(text) #tokenize the text

    #stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    text = [word for word in text if word.lower() not in stopwords] #remove word in stopwords

    #stemming & lemmatization
    newtext = []
    for j, word in enumerate(text):
        lemmword = pd.DataFrame(index=['NOUN','VERB','ADJ','ADV'], columns=['Word','Len'])    
        lemmword.loc['NOUN'] = pd.Series({'Word':lemm.lemmatize(word, wordnet.NOUN),
                                        'Len':len(lemm.lemmatize(word, wordnet.NOUN))})
        lemmword.loc['VERB'] = pd.Series({'Word':lemm.lemmatize(word, wordnet.VERB),
                                        'Len':len(lemm.lemmatize(word, wordnet.VERB))})
        lemmword.loc['ADJ'] = pd.Series({'Word':lemm.lemmatize(word, wordnet.ADJ),
                                        'Len':len(lemm.lemmatize(word, wordnet.ADJ))})
        lemmword.loc['ADV'] = pd.Series({'Word':lemm.lemmatize(word, wordnet.ADV),
                                        'Len':len(lemm.lemmatize(word, wordnet.ADV))})
        a = lemmword.loc[lemmword['Len']==lemmword['Len'].min()]['Word'][0]
        newtext.append(a)
    train['corpus'].iat[i] = ' '.join(newtext)
t1 = time.time()
total = t1-t0
print(total)
print(train.head(10))

1.1437585353851318
        id                                               text author  \
0  id26305  This process, however, afforded me no means of...    EAP   
1  id17569  It never once occurred to me that the fumbling...    HPL   
2  id11008  In his left hand was a gold snuff box, from wh...    EAP   
3  id27763  How lovely is spring As we looked from Windsor...    MWS   
4  id12958  Finding nothing else, not even gold, the Super...    HPL   
5  id22965  A youth passed in solitude, my best years spen...    MWS   
6  id09674  The astronomer, perhaps, at this point, took r...    EAP   
7  id13515        The surcingle hung in ribands from my body.    EAP   
8  id19322  I knew that you could not say to yourself 'ste...    EAP   
9  id00912  I confess that neither the structure of langua...    MWS   

                                              corpus  
0  process however afford mean ascertain dimensio...  
1              never occur fumble might mere mistake  
2  left hand gold snuff

In [10]:
#method2
import time
t0 = time.time()

from nltk.corpus import wordnet
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
from nltk.stem.wordnet import WordNetLemmatizer
lemm = WordNetLemmatizer()

#train corpus
#for i in np.arange(train.shape[0]):
for i in np.arange(10):
    text = train.text.values[i] #extract text from data set train
    text = re.sub('[^a-zA-Z0-9]', ' ', text) #remove punctuation

    #tokenize
    text = nltk.word_tokenize(text) #tokenize the text

    #stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    text = [word for word in text if word.lower() not in stopwords] #remove word in stopwords

    #stemming & lemmatization
    newtext = []               
    for j, word in enumerate(text):
        noun = lemm.lemmatize(word, wordnet.NOUN)
        verb = lemm.lemmatize(word, wordnet.VERB)
        adj = lemm.lemmatize(word, wordnet.ADJ)
        adv = lemm.lemmatize(word, wordnet.ADV)
        len_noun = len(noun)
        len_verb = len(verb)
        len_adj = len(adj)
        len_adv = len(adv)
        testing = True
    
        #len_noun is minimum
        if testing:
            if len_noun <= len_verb:
                if len_noun <= len_adj:
                    if len_noun <= len_adv:
                        newtext.append(noun)
                        testing = False

        #len_verb is minimum
        if testing:
            if len_verb <= len_noun:
                if len_verb <= len_adj:
                    if len_verb <= len_adv:
                        newtext.append(verb)
                        testing = False

        #len_adj is minimum
        if testing:
            if len_adj <= len_noun:
                if len_adj <= len_verb:
                    if len_adj <= len_adv:
                        newtext.append(adj)
                        testing = False

        #len_adv is minimum
        if testing:
            if len_adv <= len_noun:
                if len_adv <= len_verb:
                    if len_adv <= len_adj:
                        newtext.append(adv)
                        testing = False
    train['corpus'].iat[i] = ' '.join(newtext)

t1 = time.time()
total = t1-t0
print(total)
print(train.head(10))

0.02601790428161621
        id                                               text author  \
0  id26305  This process, however, afforded me no means of...    EAP   
1  id17569  It never once occurred to me that the fumbling...    HPL   
2  id11008  In his left hand was a gold snuff box, from wh...    EAP   
3  id27763  How lovely is spring As we looked from Windsor...    MWS   
4  id12958  Finding nothing else, not even gold, the Super...    HPL   
5  id22965  A youth passed in solitude, my best years spen...    MWS   
6  id09674  The astronomer, perhaps, at this point, took r...    EAP   
7  id13515        The surcingle hung in ribands from my body.    EAP   
8  id19322  I knew that you could not say to yourself 'ste...    EAP   
9  id00912  I confess that neither the structure of langua...    MWS   

                                              corpus  
0  process however afford mean ascertain dimensio...  
1              never occur fumble might mere mistake  
2  left hand gold snuf

In conclusion, two method have the same results with method 2 is around 57x faster. Next, extend method 2 to whole dataset.

In [11]:
#method2
import time
t0 = time.time()

from nltk.corpus import wordnet
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
from nltk.stem.wordnet import WordNetLemmatizer
lemm = WordNetLemmatizer()

#train corpus
for i in np.arange(train.shape[0]):
#for i in np.arange(10):
    text = train.text.values[i] #extract text from data set train
    text = re.sub('[^a-zA-Z0-9]', ' ', text) #remove punctuation

    #tokenize
    text = nltk.word_tokenize(text) #tokenize the text

    #stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    text = [word for word in text if word.lower() not in stopwords] #remove word in stopwords

    #stemming & lemmatization
    newtext = []               
    for j, word in enumerate(text):
        noun = lemm.lemmatize(word, wordnet.NOUN)
        verb = lemm.lemmatize(word, wordnet.VERB)
        adj = lemm.lemmatize(word, wordnet.ADJ)
        adv = lemm.lemmatize(word, wordnet.ADV)
        len_noun = len(noun)
        len_verb = len(verb)
        len_adj = len(adj)
        len_adv = len(adv)
        testing = True
    
        #len_noun is minimum
        if testing:
            if len_noun <= len_verb:
                if len_noun <= len_adj:
                    if len_noun <= len_adv:
                        newtext.append(noun)
                        testing = False

        #len_verb is minimum
        if testing:
            if len_verb <= len_noun:
                if len_verb <= len_adj:
                    if len_verb <= len_adv:
                        newtext.append(verb)
                        testing = False

        #len_adj is minimum
        if testing:
            if len_adj <= len_noun:
                if len_adj <= len_verb:
                    if len_adj <= len_adv:
                        newtext.append(adj)
                        testing = False

        #len_adv is minimum
        if testing:
            if len_adv <= len_noun:
                if len_adv <= len_verb:
                    if len_adv <= len_adj:
                        newtext.append(adv)
                        testing = False
    train['corpus'].iat[i] = ' '.join(newtext)

    
#test corpus
for i in np.arange(test.shape[0]):
#for i in np.arange(10):
    text = test.text.values[i] #extract text from data set train
    text = re.sub('[^a-zA-Z0-9]', ' ', text) #remove punctuation

    #tokenize
    text = nltk.word_tokenize(text) #tokenize the text

    #stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    text = [word for word in text if word.lower() not in stopwords] #remove word in stopwords

    #stemming & lemmatization
    newtext = []               
    for j, word in enumerate(text):
        noun = lemm.lemmatize(word, wordnet.NOUN)
        verb = lemm.lemmatize(word, wordnet.VERB)
        adj = lemm.lemmatize(word, wordnet.ADJ)
        adv = lemm.lemmatize(word, wordnet.ADV)
        len_noun = len(noun)
        len_verb = len(verb)
        len_adj = len(adj)
        len_adv = len(adv)
        testing = True
    
        #len_noun is minimum
        if testing:
            if len_noun <= len_verb:
                if len_noun <= len_adj:
                    if len_noun <= len_adv:
                        newtext.append(noun)
                        testing = False

        #len_verb is minimum
        if testing:
            if len_verb <= len_noun:
                if len_verb <= len_adj:
                    if len_verb <= len_adv:
                        newtext.append(verb)
                        testing = False

        #len_adj is minimum
        if testing:
            if len_adj <= len_noun:
                if len_adj <= len_verb:
                    if len_adj <= len_adv:
                        newtext.append(adj)
                        testing = False

        #len_adv is minimum
        if testing:
            if len_adv <= len_noun:
                if len_adv <= len_verb:
                    if len_adv <= len_adj:
                        newtext.append(adv)
                        testing = False
    test['corpus'].iat[i] = ' '.join(newtext)
    
t1 = time.time()
total = t1-t0
print(total)

62.920990228652954


## One Hot Encoding
One hot encoding the author columns in train dataset.

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['author'] = le.fit_transform(train['author'])

In [14]:
#the data set now looks like ....
train.head()

,id,text,author,corpus
0,id26305,"This process, however, afforded me no means of...",0,process however afford mean ascertain dimensio...
1,id17569,It never once occurred to me that the fumbling...,1,never occur fumble might mere mistake
2,id11008,"In his left hand was a gold snuff box, from wh...",0,left hand gold snuff box caper hill cut manner...
3,id27763,How lovely is spring As we looked from Windsor...,2,lovely spring look Windsor Terrace sixteen fer...
4,id12958,"Finding nothing else, not even gold, the Super...",1,Finding nothing else even gold Superintendent ...


## Build Predictive Model
We will build predictive model with:
- tfidf 
- count features
- logistic regression
- naive bayes
- grid search
- pipeline
First, we define the function to compute logloss

In [15]:
#function to compute logloss
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

**Import libraries**

In [16]:
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

C:\Users\LW130003\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


**Split train data set into training and validation data set**

In [19]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.corpus.values, train.author, 
                                                  stratify=train.author, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [20]:
print (xtrain.shape)
print (xvalid.shape)

(17621,)
(1958,)


## 1. Logistic Regression with TF-ID

In [21]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.616 


## 2. Logistic Regression with Count Vectorizer

In [23]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.537 


## 3. Simple Naive Bayes on TF-ID

In [26]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.578 


## 4. Simple Naive Bayes on Count Vectorizer

In [28]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.508 


## 5. GridSearch and Pipeline

In [32]:
classifier = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])
classifier.fit(xtrain, ytrain)

# parameter tuning with grid search
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (0, 0.01, 0.05, 0.1, 0.3, 0.5),
}
gs_clf = GridSearchCV(classifier, parameters)
gs_clf.fit(xtrain, ytrain)


# Predicting the Test set results
predictions = gs_clf.predict_proba(xvalid)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

C:\Users\LW130003\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\LW130003\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\LW130003\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\LW130003\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\LW130003\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\LW1300

logloss: 0.425 
